In [1]:
!pip install groq python-dotenv numpy tqdm datasets


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

c:\Users\da396\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [4]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello there! I'm excited to help you with any math problems you might have. Whether it's algebra, geometry, calculus, or something else, I'm here to assist you. What kind of math problem are you struggling with or would like help with?


#### GSM8K 데이터셋 확인해보기

In [5]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [6]:
import re

def extract_final_answer(response: str) -> str | None:
    
    if response is None:
        return None

    primary_match = re.search(r"####\s*([0-9,]*\.?[0-9]+)", response)
    if primary_match:
        return primary_match.group(1).replace(",", "")

    fallback_match = re.findall(r'([0-9,]+\.?[0-9]*)', response)
    if fallback_match:
        return fallback_match[-1].replace(",", "")

    return None

In [7]:
import re
from tqdm import tqdm

def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]

        try:
            correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])
        except Exception as e:
            if VERBOSE:
                print(f"[정답 파싱 실패] {e}")
            continue  # skip this sample

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)

            predicted_answer = extract_final_answer(response)

            try:
                if predicted_answer is None or str(predicted_answer).strip() == "":
                    raise ValueError("No answer extracted.")

                predicted_answer = float(str(predicted_answer).replace(",", ""))
                diff = abs(predicted_answer - correct_answer)
                is_correct = diff < 1e-5
            except Exception as e:
                if VERBOSE:
                    print(f"[예측 정답 파싱 실패] 예측값: {predicted_answer} / 에러: {e}")
                predicted_answer = None
                is_correct = False

            if is_correct:
                correct += 1
            total += 1

            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct / total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct / total if total > 0 else 0


In [8]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [9]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [10]:
### 어떤 방식으로 저장되는지 확인해보세요!
for shot in [0, 3, 5]:
    PROMPT = construct_direct_prompt(5)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )
    save_final_result(results, accuracy, f"direct_prompting_{shot}.txt")


  0%|          | 0/50 [00:00<?, ?it/s]

 10%|█         | 5/50 [00:04<00:38,  1.18it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:10<00:59,  1.50s/it]

Progress: [10/50]
Current Acc.: [20.00%]


...

Progress: [40/50]
Current Acc.: [20.00%]


 90%|█████████ | 45/50 [02:47<00:18,  3.77s/it]

Progress: [45/50]
Current Acc.: [17.78%]


100%|██████████| 50/50 [03:06<00:00,  3.72s/it]

Progress: [50/50]
Current Acc.: [20.00%]


In [11]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [12]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction:\nSolve the following mathematical question step-by-step using chain-of-thought reasoning. Provide the final numerical answer after the tag '####'.\n" #TODO: 프롬프트를 작성해주세요!

    for i in range(num_examples):
        cur_question = train_dataset['question'][sampled_indices[i]]
        cur_answer = train_dataset['answer'][sampled_indices[i]].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [14]:
for shot in [0, 3, 5]:
    PROMPT = construct_CoT_prompt(shot)
    
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    save_final_result(results, accuracy, f"CoT_prompting_{shot}.txt")


 10%|█         | 5/50 [00:04<00:37,  1.19it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:08<00:33,  1.18it/s]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:11<00:22,  1.56it/s]

...

Progress: [40/50]
Current Acc.: [72.50%]


 90%|█████████ | 45/50 [08:52<01:00, 12.01s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [09:52<00:00, 11.84s/it]

Progress: [50/50]
Current Acc.: [74.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [ ]:
def construct_best_prompt(num_examples: int = 3) -> str:
    """
    GSM8K 스타일에 최적화된, 직접 엄선한 고품질 예시를 사용한 프롬프트 생성 함수
    """
    prompt = (
        "Instruction:\nSolve the following mathematical question step-by-step using chain-of-thought reasoning. "
        "You are a world-class mathematician. Your task is to solve the following math word problem "
        "by breaking it down into simple, logical steps. Explain your reasoning for each step. "
        "Conclude with the final numerical answer on a new line, prefixed with '####'.\n\n"
        "Here are some examples:\n\n"
        "question : Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n"
        "answer : In April, Natalia sold 48 clips.\nIn May, she sold half as many as in April, so she sold 48 / 2 = <<48/2=24>>24 clips.\nTo find the total number of clips sold, we add the clips from April and May: 48 + 24 = <<48+24=72>>72 clips.\n#### 72\n\n"
        "question: A bakery has 100 kilograms of flour. It uses 1/4 of the flour for bread and 2/5 of the *remaining* flour for cakes. How many kilograms of flour are left?\n"
        "answer : The bakery starts with 100 kg of flour.\nIt uses 1/4 for bread, which is 100 * (1/4) = <<100*0.25=25>>25 kg.\nThe remaining flour is 100 - 25 = <<100-25=75>>75 kg.\nIt uses 2/5 of the remaining flour for cakes, which is 75 * (2/5) = <<75*0.4=30>>30 kg.\nThe flour left is the remainder after making cakes: 75 - 30 = <<75-30=45>>45 kg.\n#### 45\n\n"
        "Let's think step by step!"
    )

    # num_examples 샘플 추가
    sampled_indices = random.sample(range(len(train_dataset['question'])), num_examples)
    for i in range(num_examples):
        cur_question = train_dataset['question'][sampled_indices[i]]
        cur_answer = train_dataset['answer'][sampled_indices[i]].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    return prompt


In [11]:
for shot in [0, 3, 5]:
    PROMPT = construct_best_prompt(num_examples=shot)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")


TypeError: can only concatenate tuple (not "str") to tuple

In [13]:
# 5-shot만 실험하는 코드
shot = 5
PROMPT = construct_best_prompt(num_examples=shot)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)

save_final_result(results, accuracy, f"My_prompting_{shot}.txt")


 10%|█         | 5/50 [00:06<00:38,  1.18it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:28<03:08,  4.72s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [01:12<04:55,  8.44s/it]

...

Progress: [40/50]
Current Acc.: [70.00%]


 90%|█████████ | 45/50 [05:35<00:40,  8.18s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [06:19<00:00,  7.59s/it]

Progress: [50/50]
Current Acc.: [74.00%]


In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!